# Training Transformers models

<img src="https://i0.wp.com/wallur.com/wp-content/uploads/2016/12/transformers-background-1.jpg?w=1920">
<div align="right"><a href=http://wallur.com/wallpaper/36471>Image source</a></div>

In this notebook we will tackle the task of detecting toxic comments in social media, making use of a pre-trained Transformer-based language model to do so. The data we will use are a simplified version of the [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) dataset, where comments with several toxicity labels have been simplified to just one label (multilabel to multiclass).

To avoid missing packages and compatibility issues you should run this notebook under the environment defined in the accompanying environment file, or make use of [Google Colaboratory](https://colab.research.google.com/). If you use Colaboratory make sure to [activate GPU support](https://colab.research.google.com/notebooks/gpu.ipynb).

If your are running this notebook in Google Colaboratory, you will need to install the transformers library by uncommenting and running the following line.

In [1]:
#pip install transformers==2.5.1

Let us set a random seed so experiments are reproducible across runs

In [2]:
import torch
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

## Data loading

Data is provided as two separate files, one with texts for training the model and another one for testing. Both files are available in compressed form under the *data* folder.

In [3]:
import pandas as pd
train = pd.read_csv("data/toxic_multiclass_train.csv.zip", index_col="id")
test = pd.read_csv("data/toxic_multiclass_test.csv.zip", index_col="id")

If you have loaded the data properly, you should be able to visualize the first rows of each data set as follows

In [4]:
train.head(10)

,comment_text,toxicity
id,,
e0fdfd98c66fb643,"""\n\n Huggle not working \n\nHi Gurch. There i...",normal
1864753b5fb6c9a3,Mossad actually. I know where you live.,normal
ce1db53fb22d399c,REDIRECT Talk:UFC Fight Night: Belfort vs. Hen...,normal
fed4f08d59399398,"""\n\nUPA IRC\nWhat about 19:00 UTC? e | ταλκ """,normal
06e7f93938ad9e72,"""\nI've re-added your information, together wi...",normal
4a5e851879fdd674,"""\nI'm not an elitist, I'm just spreading the ...",normal
ff39db4975a78363,"""\n\nIt is not listed on this European list as...",normal
73cc03c5e157ce86,You made a mistake you ass.,obscene
ca0891e20b7bbd66,Lol dynamic IP. Just you try to stop me! 82.13...,normal


In [5]:
test.head(10)

,comment_text,toxicity
id,,
dd2dcd01f0536e53,"""\nEdit: Please stated the basis for your acc...",normal
b20b1e8381e306f8,Wikipedia is a repetition of the old joke that...,normal
54a933831401b9b2,And the same in Serbian:ђе and Croatian:če and...,normal
e7a4575dba88d9f3,"""\n\n Congrats... You gave an awesome answer i...",normal
452d5cc2ccd1d611,Wiki users controlling information \n\nDeliber...,normal
e13de405c4f2ae03,The name Tajik no more implies a lack of conne...,normal
a7b33257178c17a7,Updated? \n\nAccording to this website (http:/...,normal
d6093ce71ecbb577,"""\n\nIt is incorrect the way it sounds now. Th...",normal
7e5f6eafd2cd3147,"Template:Geobox coor \n\nHi, I had to do an em...",normal


As you can see, the data files include a column *comment_text* with the text we must classify, and an additional columns with the kind of toxicity that is presents in a comment: *toxic*, *severe_toxic*, *obscene*, *threat*, *insult* and *identity_hate*, or *normal* if the text contains no toxicity.

### Reducing the training data

To allow faster experimenting, we will only use a portion of the data. Note that reducing the training data will result in worse model performance, and reducing the test data will result in a poorer estimate of the performance of the model. If you want to obtain the best results with the best confidence, do not run the following cell. But be prepared for a very, VERY long training!

In [6]:
import numpy as np

print(f"Training patterns before reduction: {len(train)}")
train = train.sample(int(len(train)/100), random_state=12345)
print(f"Training patterns after reduction:  {len(train)}")

print(f"Test patterns before reduction: {len(test)}")
test = test.sample(int(len(test)/100), random_state=12345)
print(f"Test patterns after reduction:  {len(test)}")

Training patterns before reduction: 119678
Training patterns after reduction:  1196
Test patterns before reduction: 39893
Test patterns after reduction:  398


### Extract X and Y

In [7]:
from sklearn.preprocessing import LabelEncoder

X_train = train["comment_text"].values
X_test = test["comment_text"].values

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train["toxicity"].values)
y_test = label_encoder.transform(test["toxicity"].values)

## Transformers library

A convenient library to make use of Transformer-based language models is... [Transformers](https://github.com/huggingface/transformers)!

<img src="img/transformers_logo_name.png">

Transformers provides implementations of many language models like BERT, GPT-2 and many more. It also allows to make use of pre-trained versions of these models, thus saving a lot of time when solving practical problems.

Let's start by importing an AutoConfig object, which allows us to specify the configuration details for a language model.

In [8]:
from transformers import AutoConfig

/home/alvaro.barbero/anaconda3/envs/deeplearning-labs-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alvaro.barbero/anaconda3/envs/deeplearning-labs-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alvaro.barbero/anaconda3/envs/deeplearning-labs-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("q

Among the provided models, in this notebook we will make use of DistilBERT, a distilled version of BERT that can obtain good accuracies while keeping the model size small. We will use the configuration to tell Transformers we want to use a pretrained version of DistilBERT, trained on a dataset of uncased data, since case might not be important for the problem at hand. We also need to specify that we will use this pre-trained model to solve a classification problem with a specific number of labels.

In [9]:
pretrained_model = 'distilbert-base-uncased'
num_labels = len(set(y_train))
config = AutoConfig.from_pretrained(pretrained_model, num_labels=num_labels)

We can check out the resultant configuration, which contains all the model parameters, like dropout rates, embeddings sizes, and so on

In [10]:
config

DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

## Tokenization

The first step in a language model pipeline is to tokenize the data. We can do so using an AutoTokenizer

In [11]:
from transformers import AutoTokenizer

Again, we will load a particular tokenizer: the one used for training DistilBERT. This tokenizer is pre-trained with an uncased dataset, following the pattern we specifyied in the configuration above

In [12]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Let's check that the tokenizer works

In [13]:
tokenizer.tokenize("A long trip to Mordor")

['a', 'long', 'trip', 'to', 'mor', '##dor']

Most BERT models use a word-pieces tokenizer, dividing text into tokens that might represent a whole word, or a part of a word if the word is not common in the language. Also, since we are using an uncased model, the tokenizer maps all words to lower case.

Equivalently, we can also ask the tokenizer to transform the text to a list of dictionary ids, plus other lists of indexes required by the model.

In [14]:
tokenizer.encode("A long trip to Mordor")

[101, 1037, 2146, 4440, 2000, 22822, 7983, 102]

This encoding also adds the special tokens `[CLS]` and `[SEP]` required by BERT models at the beginning and end of each text. We can check that out as follows:

In [15]:
for token_id in tokenizer.encode("A long trip to Mordor"):
    print(f'{token_id} -> {tokenizer.decode([token_id])}')

101 -> [CLS]
1037 -> a
2146 -> long
4440 -> trip
2000 -> to
22822 -> mor
7983 -> ##dor
102 -> [SEP]


If we want to tokenize more than one text we can use the `batch_encode_plus` function. We can configure this function to make sure that every encoded text has the same length, which will we useful when working in batches on the GPU. In  the following example we will use a common length of 10, which manages to cover all tokens in every one of these sample texts.

In [16]:
texts = [
    "A long trip to Mordor", 
    "Our mind a sea",
    "Mabuka is the end of light"
]

tokenizer.batch_encode_plus(texts, max_length=10, pad_to_max_length=True)

{'input_ids': [[101, 1037, 2146, 4440, 2000, 22822, 7983, 102, 0, 0], [101, 2256, 2568, 1037, 2712, 102, 0, 0, 0, 0], [101, 26661, 15750, 2003, 1996, 2203, 1997, 2422, 102, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]}

`batch_encode_plus` returns a dictionary with three entries:

* `input_ids`: the ids of the tokens encoding each of the texts.
* `attention_mask`: 0/1 indicators telling whether the attention layers should consider this token in the mixings or not. Padding tokens always get a 0 value in the mask.
* (optional) `token_type_ids`: for language models that learn with pairs of sentences, 0/1 indicators telling the sentence to which each token belongs.

All the returned elements will be necessary as inputs to the language model. Since DistilBERT does not learn from pairs of sentences, the `token_type_ids` entry is not returned.

We can also ask the `batch_encode_plus` function to produce Tensorflow or Pytorch tensors instead of python lists. For instance, to obtain Pytorch tensors we will do as follows:

In [17]:
tokenizer.batch_encode_plus(texts, max_length=10, pad_to_max_length=True, return_tensors="pt")

{'input_ids': tensor([[  101,  1037,  2146,  4440,  2000, 22822,  7983,   102,     0,     0],
        [  101,  2256,  2568,  1037,  2712,   102,     0,     0,     0,     0],
        [  101, 26661, 15750,  2003,  1996,  2203,  1997,  2422,   102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

The returned structure is the same, but now each entry is a Pytorch tensor.

Now, what would be the ideal maximum length for encoding our texts? BERT accepts inputs texts as long as 512 tokens, but using always this maximum length will result in slow training times. We can try tokenizing all texts without length limitation and study the distribution of text lengths.

<img src="img/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Perform the following tasks to find an appropriate maximum length to use in the encoding:
    <ul>
        <li>Encode all texts in X_train using batch_encode_plus, but don't specify any maximum length, padding or tensor options.</li>
        <li>Create a list with the lengths of each one of the indices vectors.</li>
        <li>Find the value of 90% percentile of the lengths distribution. Tip: use the <a href=https://docs.scipy.org/doc/numpy/reference/generated/numpy.quantile.html>numpy quantile function</a>. Save this value as an integer number into a variable named <b>maxlength</b>.</li>
    </ul>
</font>

***

In [18]:
####### INSERT YOUR CODE HERE
import numpy as np

encoded = tokenizer.batch_encode_plus(X_train)
lenghts = [len(x) for x in encoded["input_ids"]]
maxlength = int(np.quantile(lenghts, 0.9))
maxlength

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (592 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (630 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (600 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

219

We will use this maximum length later on.

## DistilBERT model

We are now ready to explore the DistilBERT model. Let's load the pre-trained version of DistilBERT using an AutoModel class

In [19]:
from transformers import AutoModel

distilbert = AutoModel.from_pretrained('distilbert-base-uncased')

This pre-trained version contains the "body" of the model, which can receive a sequence of tokens and produce the "contextualized" embeddings for each one of those tokens.

<img src="http://jalammar.github.io/images/bert-encoders-input.png">
<div align="right">Image credit: <a href="http://jalammar.github.io/illustrated-bert/">The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)</a></div>

We can try this with a batch of one text of the training data, but remember we first need to transform it through the tokenizer and obtain Pytorch tensors

In [20]:
sample = tokenizer.batch_encode_plus(X_train[0:1], max_length=40, pad_to_max_length=True, return_tensors="pt")
sample

{'input_ids': tensor([[  101,  1000,  3835,  2862,   999,  1027,  1027, 19394,  2015,  2006,
          4526,  2862,  1997, 27672,  6244,  5231,  1999,  1996,  2142,  2163,
          1010,  2029,  2038,  2000,  2022,  2028,  1997,  1996,  6493,  2862,
          1045,  2031,  2412,  2464,  2006,  2019,  3988,  3720,  4325,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Now we can input the tensor into the DistilBERT model. A convenient way to input all three tensors into the model is using the unpacking operator `**`

In [21]:
outputs = distilbert(**sample)

A Transformers model always returns a tuple which might contain several pieces of information. In the case of DistilBERT, only a single object is returned, which is a pytorch tensor containing the embeddings

In [22]:
embeddings = outputs[0]
print(f"Input tensor shape {sample['input_ids'].shape}")
print(f"Input tensor values {sample['input_ids']}")
print(f"DistilBERT embeddings shape {embeddings.shape}")
print(f"DistilBERT embeddings values {embeddings}")

Input tensor shape torch.Size([1, 40])
Input tensor values tensor([[  101,  1000,  3835,  2862,   999,  1027,  1027, 19394,  2015,  2006,
          4526,  2862,  1997, 27672,  6244,  5231,  1999,  1996,  2142,  2163,
          1010,  2029,  2038,  2000,  2022,  2028,  1997,  1996,  6493,  2862,
          1045,  2031,  2412,  2464,  2006,  2019,  3988,  3720,  4325,   102]])
DistilBERT embeddings shape torch.Size([1, 40, 768])
DistilBERT embeddings values tensor([[[ 0.0868,  0.1375, -0.1329,  ...,  0.0667,  0.4648,  0.4608],
         [ 0.3281,  0.1915,  0.0478,  ...,  0.1433,  0.4600,  0.2365],
         [ 0.3558,  0.1741,  0.4733,  ...,  0.1318,  0.2100,  0.1237],
         ...,
         [ 0.4565, -0.1248,  0.2463,  ...,  0.1870,  0.0948, -0.0672],
         [ 0.1072, -0.0313,  0.0774,  ...,  0.1644,  0.2107, -0.2898],
         [ 0.7191,  0.2547, -0.1020,  ...,  0.2076, -0.6422, -0.2012]]],
       grad_fn=<NativeLayerNormBackward>)


DistilBERT returns an embedding vector of 768 numbers for each input token.

Although it is tempting to use these embeddings as features for the toxic classification task, this approach does not generally give good results. Instead, it is advisable to add a classification "head" to the model, growing out of the embedding produced for the `[CLS]` special token, and fine-tune the whole model to the task through back-propagation.

<img src="http://jalammar.github.io/images/bert-classifier.png">
<div align="right">Image credit: <a href="http://jalammar.github.io/illustrated-bert/">The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)</a></div>

The Transformers library can prepare all of this for us, by loading a version of DistilBERT with a Sequence Classification head. We will provide the configuration we prepared above, so the the classification head produces as many outputs as classes in our problem.

In [23]:
from transformers import AutoModelForSequenceClassification
distilbert_classification = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)

## Preparing for fine-tuning

Fine-tuning a Transformers model is not as simple as fitting scikit-learn or Keras model: we will need to provide all the details on how to batch the data, as well as other details on the training procedure. To easen this task, we will first prepare some useful functions.

### Using the GPU

First, fine-tuning a language model on the CPU is not a good idea. We will be better off using a GPU. To do so, we first need to identify the computing device. The code below checks if a GPU is available in the system, and if so, prepares a Torch device to send the calculations there.

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

The cell above should print `device(type='cuda')` if a GPU was found.

Note that if your machine has several GPUs, by default the first one will be used, but the underlying deep learning library (Tensorflow or Pytorch) might lock the memory from all GPUs in the system. To constrain this notebook to use a specific GPU, run jupyter notebook as 

   `CUDA_VISIBLE_DEVICES=X jupyter notebook`
   
where `X` is the number of the GPU you want to use. You can check your available GPUs and their usage with the following command:

In [25]:
!nvidia-smi

Thu Jun 11 13:51:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.64       Driver Version: 430.64       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K20m          Off  | 00000000:85:00.0 Off |                    0 |
| N/A   37C    P0    47W / 225W |    232MiB /  4743MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K20m          Off  | 00000000:86:00.0 Off |                    0 |
| N/A   37C    P0    47W / 225W |     69MiB /  4743MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

### Datasets

Any dataset used in Transformers must follow one contraint: the dataset must be an iterable of samples. But what is a sample? A text together with the outputs we expect our model to generate for it. In our classification problem each sample will be a text together with its class, so a natural way to organize a sample is the forma a tuple (text, class). We will prepare out training and test data in this format:

In [26]:
train_dataset = list(zip(X_train, y_train))
eval_dataset = list(zip(X_test, y_test))

### Collator

Now, we will need a <b>collate class</b> that receives an part of a dataset (iterable of samples), and performs all the tokenization and encoding procedure to obtain a Torch tensor in GPU. This class should inherit from the Transformers `DataCollator` class, and implement a `collate_batch` method that receives an iterable of samples and returns an encoded batch. Here we provide an implementation for you:

In [27]:
from transformers import DataCollator

class TextClassificationCollator(DataCollator):
    """Data collator for a text classification problem"""
    
    def __init__(self, tokenizer, max_length):
        """Initializes the collator with a tokenizer and a maximum document length (in tokens)"""
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def encode_texts(self, texts):
        """Transforms an iterable of texts into a dictionary of model input tensors, stored in the GPU"""
        # Tokenize and encode texts as tensors, with maximum length
        tensors = self.tokenizer.batch_encode_plus(
            texts, 
            max_length=self.max_length, 
            pad_to_max_length=True, 
            return_tensors="pt"
        )
        # Move tensors to GPU
        for key in tensors:
            tensors[key] = tensors[key].to(device)
        return tensors
    
    def collate_batch(self, patterns):
        """Collate a batch of patterns
        
        Arguments:
            - patterns: iterable of tuples in the form (text, class)
            
        Output: dictionary of torch tensors ready for model input
        """
        # Split texts and classes from the input list of tuples
        train_idx, targets = zip(*patterns)
        # Encode inputs
        input_tensors = self.encode_texts(train_idx)
        # Transform class labels to a tensor in GPU
        Y = torch.tensor(targets).long().to(device)
        # Return batch as a dictionary wikth all the inputs tensors and the labels
        batch = {**input_tensors, "labels": Y}
        return batch

With the collator class defined, we will create an instance for the particular tokenizer and maximum sequence length we have chosen above

In [28]:
collator = TextClassificationCollator(tokenizer, maxlength)

### Training arguments

The next step is creating a `TrainingArguments` object. This object allows us to specify the training procedure details. For this notebook we will use a batch size of 32, which should fit into a small GPU. We will also train the model only 1 epoch over the training data, to allow us to check the results quickly.

In [29]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./models/toxic_model",  # Folder in which to save the trained model
    overwrite_output_dir=True,  # Whether to overwrite previous models found in the output folder
    per_gpu_train_batch_size=16,  # batch size during training
    per_gpu_eval_batch_size=128,  # batch size during evaluation (prediction)
    num_train_epochs=1,  # Model training epochs
    logging_steps=25,  # After how many training steps (batches) a log message showing progress will be printed
    save_steps=25  # After how many training steps (batches) the model will be checkpointed to disk
)

### Trainer

The last step of the preparation, is to create a `Trainer` object. This is the object that performs the actual training, and will need to receive all the information we prepared above, which is:

* The model to be fine-tuned
* The `TrainingArguments` object we prepared above
* Training and evaluation datasets
* The DataCollator object that will batch the data.

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=distilbert_classification,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collator
)

Everything is ready for training!

## Model training and evaluation

Once all the work above is done, training is easy

In [31]:
%%time
trainer.train()

{"loss": 0.8196162295341491, "learning_rate": 3.3333333333333335e-05, "epoch": 0.3333333333333333, "step": 25}
{"loss": 0.3887899678945541, "learning_rate": 1.6666666666666667e-05, "epoch": 0.6666666666666666, "step": 50}
{"loss": 0.41151548027992246, "learning_rate": 0.0, "epoch": 1.0, "step": 75}


CPU times: user 37.1 s, sys: 17.1 s, total: 54.2 s
Wall time: 1min 34s


TrainOutput(global_step=75, training_loss=0.539973892569542)

Once the model is trained we can obtain the model predictions with the `predict` method of the `Trainer` object

In [32]:
preds = trainer.predict(eval_dataset)

We can see that in a `ForSequenceClassification` model each prediction is a list of **unnormalized** probabilities for each class.

In [33]:
preds.predictions[0]

array([-0.7314742 ,  0.03136623,  4.298269  , -0.98842025, -1.3786166 ,
       -1.3473176 ,  0.37629357], dtype=float32)

To obtain actual probabilities we need to apply a `softmax` function, which enforces each probability to take a value in the range `[0, 1]`, and also that all probabilities sum up to `1`.

In [34]:
from scipy.special import softmax
probs = softmax(preds.predictions, axis=1)

We can check now that indeed the number we have look like probabilities 

In [35]:
probs[0]

array([0.00621493, 0.01332705, 0.95022416, 0.00480669, 0.00325377,
       0.00335721, 0.01881629], dtype=float32)

Now we can use these probabilities to compute any standard metric from scikit-learn. For instance, ROC AUC

In [36]:
from sklearn.metrics import roc_auc_score
print("AUC score", roc_auc_score(y_test, probs, multi_class='ovr'))

AUC score 0.9055487893001456
